In [2]:
import modulos as md
lista=['THETAUSDT']
lista=md.lista_de_monedas()

for symbol in lista:
    try:
        data = md.obtiene_historial(symbol,'30m')
        data = md.estrategia_santa(data)
        resultado = md.backtesting(data,tp_flag=False,plot_flag=False)
        if resultado[6]>1.5:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]}")
    except Exception as falla:
        pass  


EOSUSDT - retorno: 6.15% - trades: 13
XLMUSDT - retorno: 2.13% - trades: 9
XTZUSDT - retorno: 4.03% - trades: 7
ONTUSDT - retorno: 2.77% - trades: 6
IOTAUSDT - retorno: 1.91% - trades: 9
BATUSDT - retorno: 3.29% - trades: 6
THETAUSDT - retorno: 9.33% - trades: 6
KNCUSDT - retorno: 7.47% - trades: 6
ZRXUSDT - retorno: 3.22% - trades: 9
OMGUSDT - retorno: 11.38% - trades: 7
SXPUSDT - retorno: 2.64% - trades: 8
KAVAUSDT - retorno: 15.97% - trades: 8


c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)



Error: APIError(code=-1121): Invalid symbol. - line: 236 - file: modulos.py - par: DEFIUSDT



c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:313: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


CRVUSDT - retorno: 17.03% - trades: 6
RUNEUSDT - retorno: 21.17% - trades: 4
SUSHIUSDT - retorno: 8.32% - trades: 6
SRMUSDT - retorno: 22.43% - trades: 12
ICXUSDT - retorno: 2.47% - trades: 8
STORJUSDT - retorno: 24.28% - trades: 8
FTMUSDT - retorno: 4.1% - trades: 7
ENJUSDT - retorno: 4.73% - trades: 7
FLMUSDT - retorno: 1.79% - trades: 12
LRCUSDT - retorno: 2.19% - trades: 7
MATICUSDT - retorno: 1.96% - trades: 11
OCEANUSDT - retorno: 1.57% - trades: 6
BELUSDT - retorno: 9.68% - trades: 6
GRTUSDT - retorno: 2.23% - trades: 8
1INCHUSDT - retorno: 4.96% - trades: 5
SANDUSDT - retorno: 3.84% - trades: 7
LITUSDT - retorno: 9.8% - trades: 9
SFPUSDT - retorno: 11.81% - trades: 4
MANAUSDT - retorno: 4.36% - trades: 7
ALICEUSDT - retorno: 18.73% - trades: 5

Error: APIError(code=-1121): Invalid symbol. - line: 236 - file: modulos.py - par: 1000SHIBUSDT

BAKEUSDT - retorno: 1.67% - trades: 7
GTCUSDT - retorno: 17.5% - trades: 6
RAYUSDT - retorno: 2.4% - trades: 5
C98USDT - retorno: 2.72% - tr

In [2]:
import modulos as md
lista=['COMBOUSDT']
#lista=md.lista_de_monedas()

for symbol in lista:
    try:
        data = md.obtiene_historial(symbol,'1m')
        data = md.estrategia_santa(data)
        resultado = md.backtesting(data,tp_flag=False,plot_flag=False)
        if resultado[6]>1.5:
            print(f"{symbol} - retorno: {md.truncate(resultado[6],2)}% - trades: {resultado[17]}")
    except Exception as falla:
        pass  

COMBOUSDT - retorno: 10.4% - trades: 2
